In [1]:
import pandas as pd

In [2]:
anime = pd.read_csv('./mangaki-data-challenge-0908/titles.csv')
anime.head()

,work_id,title,category
0,0,Hayate the Combat Butler! Cuties,anime
1,1,Caffe Latte Rhapsody,manga
2,2,Dragon Ball,anime
3,3,Accel World,manga
4,4,Soukyuu no Fafner: Right of Left,anime


In [3]:
anime.groupby('category').count()

,work_id,title
category,,
album,1,1
anime,7088,7088
manga,2808,2808


In [4]:
#anime = anime['category'].isin(['anime', 'manga'])

In [5]:
anime.tail()

,work_id,title,category
9892,9892,Starship Operators,anime
9893,9893,Project: City Flying in the Sky,anime
9894,9894,Coeur de sirene,manga
9895,9895,Hayate no Gotoku! OVA,anime
9896,9896,"Say ""I Love You"". Specials",anime


# User ratings

In [6]:
ratings = pd.read_csv('./mangaki-data-challenge-0908/watched.csv')
ratings.head()

,user_id,work_id,rating
0,717,8025,dislike
1,1106,1027,neutral
2,1970,3949,neutral
3,1685,9815,like
4,1703,3482,like


In [7]:
ratings = ratings.replace({'dislike': '1'}, regex=True)
ratings = ratings.replace({'neutral': '2'}, regex=True)
ratings = ratings.replace({'like': '3'}, regex=True)
ratings = ratings.replace({'love': '4'}, regex=True)

ratings['rating'] = ratings['rating'].astype(float)
ratings['rating'] = ratings['rating'] / 4.0

In [8]:
ratings.head()

,user_id,work_id,rating
0,717,8025,0.25
1,1106,1027,0.50
2,1970,3949,0.50
3,1685,9815,0.75
4,1703,3482,0.75


In [9]:
ratings.sort_values('work_id').groupby('work_id').first().tail()

,user_id,rating
work_id,,
9891,877,0.75
9892,152,0.75
9893,1522,0.75
9894,985,0.25
9896,1789,0.50


In [10]:
print('Number of users', len(ratings.groupby('user_id').first()))

Number of users 1963


In [11]:
# average rating for works
ratings.groupby('work_id').mean()['rating']

work_id
0       0.500000
1       0.750000
2       0.684426
3       0.700000
4       0.642857
5       0.683036
6       0.500000
7       0.666667
9       0.593750
10      0.750000
11      0.750000
12      0.750000
16      0.583333
17      0.642857
18      0.607143
20      0.250000
21      0.687500
22      0.500000
23      0.590909
24      0.652174
25      0.375000
26      0.671875
27      0.662534
28      0.690217
29      0.666667
30      0.750000
32      0.750000
33      0.500000
34      0.500000
35      0.500000
          ...   
9864    0.833333
9865    0.700000
9866    0.750000
9867    0.750000
9868    0.639744
9869    0.601190
9870    0.750000
9871    0.500000
9872    0.500000
9873    0.500000
9874    0.250000
9875    0.666667
9876    0.666667
9877    0.615385
9878    0.500000
9879    0.812500
9880    0.750000
9881    0.833333
9882    0.750000
9884    0.562500
9885    0.725000
9886    0.617647
9887    0.650000
9889    0.750000
9890    1.000000
9891    0.750000
9892    0.687500
9893  

## Adding 0 ratings for non rated works


In [12]:
dummy_ratings = []

for work_id in range(len(anime)):
    # if not rated by any user
    if ratings[ratings['work_id'] == work_id].empty:
        dummy_ratings.append({'work_id': int(work_id), 'rating': 0, 'user_id': 0})

ratings = ratings.append(pd.DataFrame(dummy_ratings), ignore_index=True)

In [13]:
print(len(anime))
ratings.sort_values('work_id').groupby('work_id').first().tail()

9897


,rating,user_id
work_id,,
9892,0.75,1054
9893,0.75,1522
9894,0.25,985
9895,0.00,0
9896,0.50,1789


In [14]:
piv = ratings.pivot_table(index=['user_id'], columns=['work_id'], values='rating')
piv.fillna(0, inplace=True)
piv.head()

work_id,0,1,2,3,4,5,6,7,8,9,...,9887,9888,9889,9890,9891,9892,9893,9894,9895,9896
user_id,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.75,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.50,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.75,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Training Deep AutoEncoders for Collaborative Filtering

In [1]:
import argparse
import sys

import tensorflow as tf

config = tf.ConfigProto()
config.gpu_options.allow_growth=True

from tensorflow.python.estimator.model_fn import EstimatorSpec

In [2]:
tf.__version__

'1.4.0-rc1'

In [16]:
FLAGS = None

tf.logging.set_verbosity(tf.logging.INFO)

In [20]:
LEARNING_RATE = 0.005
DROPOUT = 0.8
BATCH_SIZE = 64
OPTIMIZER = tf.train.AdagradOptimizer #tf.train.GradientDescentOptimizer

LOGDIR = './model_files/deep_ae/adagrad_64_0.005_0.8_64_128_128_128_64_dense_re_last_activation'

In [21]:
def model_fn(features, labels, mode, params):
    # 1. Configure the model via TensorFlow operations
    # Input Layer
    x = features['x']

    encoder1 = tf.layers.dense(inputs=x, units=64, activation=tf.nn.selu, name='encoder1')
    encoder2 = tf.layers.dense(inputs=encoder1, units=128, activation=tf.nn.selu, name='encoder2')

    bottleneck = tf.layers.dense(inputs=encoder2, units=128, name='bottleneck')
    dropout = tf.layers.dropout(inputs=bottleneck, rate=params['dropout'], training=mode == tf.estimator.ModeKeys.TRAIN)

    decoder1 = tf.layers.dense(inputs=dropout, units=128, activation=tf.nn.selu, name='decoder1')
    decoder2 = tf.layers.dense(inputs=decoder1, units=64, activation=tf.nn.selu, name='decoder2')

    y = tf.layers.dense(inputs=decoder2, units=x.get_shape()[1], activation=tf.nn.selu, name="out")

     # Dense re-feeding in the same pass (not two optimization step per iteration)
    encoder1_re = tf.layers.dense(inputs=y, units=64, activation=tf.nn.selu, reuse=True, name='encoder1')
    encoder2_re = tf.layers.dense(inputs=encoder1_re, units=128, activation=tf.nn.selu, reuse=True, name='encoder2')

    bottleneck_re = tf.layers.dense(inputs=encoder2_re, units=128, reuse=True, name='bottleneck')
    dropout_re = tf.layers.dropout(inputs=bottleneck_re, rate=params['dropout'], training=mode == tf.estimator.ModeKeys.TRAIN)

    decoder1_re = tf.layers.dense(inputs=dropout_re, units=128, activation=tf.nn.selu, reuse=True, name='decoder1')
    decoder2_re = tf.layers.dense(inputs=decoder1_re, units=64, activation=tf.nn.selu, reuse=True, name='decoder2')

    y_re = tf.layers.dense(inputs=decoder2_re, units=x.get_shape()[1], reuse=True, activation=tf.nn.selu, name="out")
# FIXME sigmoid activation??

    # Loss function for training/evaluation
    # Masked Mean Square Error 
    # mask = tf.where(x != 0, 1, 0, name='loss_mask')
    # loss = tf.divide(tf.multiply(mask, tf.squared_difference(x, y)), tf.sum(mask)) +
    #        tf.losses.mean_squared_error(y, y_re)


    weight = tf.cast(tf.greater(x, 0), tf.float32)
    # loss for first backprop + loss for dense re-feeding
    loss = tf.losses.mean_squared_error(x, y, weight) + tf.losses.mean_squared_error(y, y_re)
    
    tf.summary.scalar("loss", loss)
    tf.summary.scalar("rmse", tf.sqrt(loss))

    eval_metric_ops = {
        "rmse": tf.metrics.root_mean_squared_error(x, y, weight)
    }
    # 3. Define the training operation/optimizer
    optimizer = params['optimizer'](learning_rate=params["learning_rate"])
    train_op = optimizer.minimize(loss=loss, global_step=tf.train.get_global_step())

    # 4. Generate predictions
    # 5. Return predictions/loss/train_op/eval_metric_ops in EstimatorSpec object
    return EstimatorSpec(mode, y, loss, train_op, eval_metric_ops)


In [22]:
# Set model params
model_params = {
    "learning_rate": LEARNING_RATE,
    "dropout": DROPOUT,
    "optimizer": OPTIMIZER
}

# Instantiate Estimator
nn = tf.estimator.Estimator(model_fn=model_fn, params=model_params, model_dir=LOGDIR, 
                            config=tf.estimator.RunConfig(session_config=config))

train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={'x': piv.as_matrix()},
    batch_size=BATCH_SIZE,
    num_epochs=None,
    shuffle=True,
    queue_capacity=10000,
    num_threads=1,
)
summary_hook = tf.train.SummarySaverHook(
    save_secs=2,
    output_dir=LOGDIR,
    summary_op=tf.summary.merge_all(),
    scaffold=tf.train.Scaffold()
)
# Train
nn.train(input_fn=train_input_fn, steps=30000, hooks=[summary_hook])  #

# Score accuracy
test_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={'x': piv.as_matrix()},  # TODO test data
    num_epochs=1,
    shuffle=False)

# ev = nn.evaluate(input_fn=test_input_fn)
# print("Loss: %s" % ev["loss"])
# print("Root Mean Squared Error: %s" % ev["rmse"])

INFO:tensorflow:Using config: {'_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_tf_random_seed': 1, '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': './model_files/deep_ae/adagrad_64_0.005_0.8_64_128_128_128_64_dense_re_last_activation', '_session_config': gpu_options {
  allow_growth: true
}
, '_log_step_count_steps': 100, '_keep_checkpoint_max': 5}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into ./model_files/deep_ae/adagrad_64_0.005_0.8_64_128_128_128_64_dense_re_last_activation/model.ckpt.
INFO:tensorflow:loss = 0.496263, step = 1
INFO:tensorflow:global_step/sec: 90.7456
INFO:tensorflow:loss = 0.4647, step = 101 (1.103 sec)
INFO:tensorflow:global_step/sec: 103.375
INFO:tensorflow:loss = 0.450493, step = 201 (0.967 sec)
INFO:tensorflow:global_step/sec: 103.662
INFO:tensorflow:loss = 0.372682, step = 301 (0.965 sec)
INFO:tensorflow:global_step/sec: 103.678
INFO:tensorflow:loss = 0.223262, ste

INFO:tensorflow:global_step/sec: 102.429
INFO:tensorflow:loss = 0.0497101, step = 7801 (0.976 sec)
INFO:tensorflow:global_step/sec: 103.396
INFO:tensorflow:loss = 0.0438645, step = 7901 (0.967 sec)
INFO:tensorflow:global_step/sec: 102.65
INFO:tensorflow:loss = 0.045565, step = 8001 (0.974 sec)
INFO:tensorflow:global_step/sec: 102.628
INFO:tensorflow:loss = 0.0394275, step = 8101 (0.974 sec)
INFO:tensorflow:global_step/sec: 103.552
INFO:tensorflow:loss = 0.0442589, step = 8201 (0.966 sec)
INFO:tensorflow:global_step/sec: 103.345
INFO:tensorflow:loss = 0.0470038, step = 8301 (0.968 sec)
INFO:tensorflow:global_step/sec: 103.178
INFO:tensorflow:loss = 0.0475958, step = 8401 (0.969 sec)
INFO:tensorflow:global_step/sec: 102.869
INFO:tensorflow:loss = 0.0413456, step = 8501 (0.972 sec)
INFO:tensorflow:global_step/sec: 102.69
INFO:tensorflow:loss = 0.0395053, step = 8601 (0.974 sec)
INFO:tensorflow:global_step/sec: 103.214
INFO:tensorflow:loss = 0.0444458, step = 8701 (0.969 sec)
INFO:tensorfl

INFO:tensorflow:global_step/sec: 103.004
INFO:tensorflow:loss = 0.0371818, step = 16101 (0.971 sec)
INFO:tensorflow:global_step/sec: 102.659
INFO:tensorflow:loss = 0.0362574, step = 16201 (0.974 sec)
INFO:tensorflow:global_step/sec: 102.86
INFO:tensorflow:loss = 0.0296875, step = 16301 (0.972 sec)
INFO:tensorflow:global_step/sec: 103.008
INFO:tensorflow:loss = 0.0342949, step = 16401 (0.971 sec)
INFO:tensorflow:global_step/sec: 102.475
INFO:tensorflow:loss = 0.0410287, step = 16501 (0.976 sec)
INFO:tensorflow:global_step/sec: 103.234
INFO:tensorflow:loss = 0.0324789, step = 16601 (0.969 sec)
INFO:tensorflow:global_step/sec: 103.576
INFO:tensorflow:loss = 0.0347208, step = 16701 (0.965 sec)
INFO:tensorflow:global_step/sec: 102.876
INFO:tensorflow:loss = 0.0365158, step = 16801 (0.972 sec)
INFO:tensorflow:global_step/sec: 103.014
INFO:tensorflow:loss = 0.0377228, step = 16901 (0.971 sec)
INFO:tensorflow:global_step/sec: 102.741
INFO:tensorflow:loss = 0.0394961, step = 17001 (0.973 sec)
I

INFO:tensorflow:loss = 0.0302188, step = 24301 (0.970 sec)
INFO:tensorflow:global_step/sec: 102.784
INFO:tensorflow:loss = 0.0332318, step = 24401 (0.973 sec)
INFO:tensorflow:global_step/sec: 103.576
INFO:tensorflow:loss = 0.03252, step = 24501 (0.965 sec)
INFO:tensorflow:global_step/sec: 103.371
INFO:tensorflow:loss = 0.0303276, step = 24601 (0.967 sec)
INFO:tensorflow:global_step/sec: 102.372
INFO:tensorflow:loss = 0.0369382, step = 24701 (0.977 sec)
INFO:tensorflow:global_step/sec: 102.94
INFO:tensorflow:loss = 0.0333401, step = 24801 (0.971 sec)
INFO:tensorflow:global_step/sec: 103.219
INFO:tensorflow:loss = 0.0319695, step = 24901 (0.969 sec)
INFO:tensorflow:global_step/sec: 103.045
INFO:tensorflow:loss = 0.036047, step = 25001 (0.970 sec)
INFO:tensorflow:global_step/sec: 102.958
INFO:tensorflow:loss = 0.0343967, step = 25101 (0.971 sec)
INFO:tensorflow:global_step/sec: 103.287
INFO:tensorflow:loss = 0.0303696, step = 25201 (0.968 sec)
INFO:tensorflow:global_step/sec: 103.212
INFO

KeyboardInterrupt: 

## Predict


### Making the prediction

In [ ]:
predict_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": piv.as_matrix()},
    num_epochs=1,
    shuffle=False)

predictions_gen = nn.predict(input_fn=predict_input_fn)

### Calculating Indexes

In [ ]:
index = piv.index.values
reverse_index = {}

for i, j in enumerate(index):
    reverse_index[j] = i

In [ ]:
print(len(index))
print(len(reverse_index))

In [ ]:
index

In [ ]:
reverse_index

In [ ]:
predictions = list(predictions_gen)
predictions

In [ ]:
test = pd.read_csv('./mangaki-data-challenge-0908/test.csv')
test.head()

In [ ]:
def get_prediction(user_id, work_id):
    try:
        row_num = reverse_index[user_id]
    except KeyError: # check if user has no ratings yet and submit global score
        return ratings.groupby('work_id').mean()['rating'][work_id]
#    print('user_id', row['user_id'], 'index', row_num, 'work', work_id)
    return predictions[row_num][work_id]

In [ ]:
test['out'] = test.apply(lambda row: get_prediction(int(row['user_id']), row['work_id']), axis=1)

In [ ]:
test.head()

In [ ]:
test.to_csv('./deep_ae1_out_adagrad_0.005_64_0.8_64_128_densere.csv', index=False)

# Collaborative Filtering On Anime Data

### Based on https://www.kaggle.com/ajmichelutti/collaborative-filtering-on-anime-data

In [ ]:
import numpy as np
import scipy as sp
import scipy.sparse

from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
# find works with no rating

In [ ]:
# Note: As we are subtracting the mean from each rating to standardize
# all users with only one rating or who had rated everything the same will be dropped

# Normalize the values
#piv_norm = piv.apply(lambda x: (x-np.mean(x))/(np.max(x)-np.min(x)), axis=1)
piv_norm = piv.apply(lambda x: x / 5, axis=1)


# Drop all columns containing only zeros representing users who did not rate
piv_norm.fillna(0, inplace=True)
piv_norm = piv_norm.T
piv_norm = piv_norm.loc[:, (piv_norm != 0).any(axis=0)]

In [ ]:
# Our data needs to be in a sparse matrix format to be read by the following functions

piv_sparse = sp.sparse.csr_matrix(piv_norm.values)

In [ ]:
item_similarity = cosine_similarity(piv_sparse)
user_similarity = cosine_similarity(piv_sparse.T)

In [ ]:
# Inserting the similarity matricies into dataframe objects

item_sim_df = pd.DataFrame(item_similarity, index = piv_norm.index, columns = piv_norm.index)
user_sim_df = pd.DataFrame(user_similarity, index = piv_norm.columns, columns = piv_norm.columns)

In [ ]:
user_sim_df.head()

In [ ]:
item_sim_df.head()

In [ ]:
# writing output
test = pd.read_csv('./mangaki-data-challenge-0908/test.csv')
test.head()

In [ ]:
# This function calculates the weighted average of similar users
# to determine a potential rating for an input user and show

def predicted_rating(anime_name, user):
    sim_users = user_sim_df.sort_values(by=user, ascending=False).index[1:1000]
    user_values = user_sim_df.sort_values(by=user, ascending=False).loc[:,user].tolist()[1:1000]
    

In [ ]:
#out = [predicted_rating(row['work_id'], row['user_id']) for i, row in test.iterrows()]

test['out'] = ratings.apply(lambda row: predicted_rating(row['work_id'], row['user_id']), axis=1)



# Adding MAL